In [ ]:
from chronos import ChronosPipeline


In [ ]:
pipeline = ChronosPipeline.from_pretrained("FineTraining/", device_map="cpu")


The module name  (originally ) is not a valid Python identifier. Please rename the original module to avoid import issues.


In [6]:
import torch
import pandas as pd

In [26]:

df=pd.read_csv("../data/eco2mix_Data.csv",index_col="Unnamed: 0")
df_train=df

In [27]:
context = torch.tensor(df_train["target"].values[-512:])

In [28]:
forecast = pipeline.predict(context, prediction_length=48, num_samples=20)

In [30]:
import numpy as np

low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)
last_ts = pd.to_datetime(df_train["Date - Heure"].iloc[-1])
future_dates = pd.date_range(start=last_ts, periods=49, freq="H")[1:]

df_pred = pd.DataFrame({
    "timestamp": future_dates,
    "prediction": median,
    "low": low,
    "high_90": high
})

print(df_pred)


                   timestamp    prediction           low       high_90
0  2025-11-09 21:00:00+01:00  47088.367188  46567.085938  47609.652344
1  2025-11-09 22:00:00+01:00  46914.621094  46567.085938  47957.144531
2  2025-11-09 23:00:00+01:00  47262.113281  46567.085938  48304.683594
3  2025-11-10 00:00:00+01:00  45524.515625  44794.735156  46567.085938
4  2025-11-10 01:00:00+01:00  43091.933594  42362.148438  43786.960938
5  2025-11-10 02:00:00+01:00  42049.363281  41319.582812  42744.394531
6  2025-11-10 03:00:00+01:00  40311.812500  39616.781250  41006.839844
7  2025-11-10 04:00:00+01:00  39964.273438  39234.492969  40346.561719
8  2025-11-10 05:00:00+01:00  42049.363281  41632.367969  42744.394531
9  2025-11-10 06:00:00+01:00  46914.621094  46150.043750  47644.401563
10 2025-11-10 07:00:00+01:00  52127.328125  51328.044531  52474.867188
11 2025-11-10 08:00:00+01:00  54212.417969  53830.172266  54559.957031
12 2025-11-10 09:00:00+01:00  54733.703125  54525.203125  55602.480469
13 202

C:\Users\Abdessamad\AppData\Local\Temp\ipykernel_5700\183207746.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=last_ts, periods=49, freq="H")[1:]


In [31]:
# Exemple : df_pred avec colonnes timestamp, prediction, high_90
df_pred.to_json("../data/data.json", orient="records", date_format="iso", indent=2)
